In [ ]:
import os
import time
import re
import json
import requests
import yfinance as yf
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import concurrent.futures

# --- Fonctions de scraping des tickers ---

def scrape_american_etf_tickers():
    url = "https://en.wikipedia.org/wiki/List_of_American_exchange-traded_funds"
    print("Scraping American ETF tickers from:", url)
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")
    tickers = set()
    pattern = re.compile(r'^[A-Z]{2,6}$')
    for a in soup.find_all('a', href=True):
        text = a.get_text(strip=True)
        if pattern.match(text):
            tickers.add(text)
    return list(tickers)

def scrape_canadian_etf_tickers():
    url = "https://en.wikipedia.org/wiki/List_of_Canadian_exchange-traded_funds"
    print("Scraping Canadian ETF tickers from:", url)
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")
    tickers = set()
    pattern = re.compile(r'^[A-Z]{2,6}(\.TO)?$')
    for a in soup.find_all('a', href=True):
        text = a.get_text(strip=True)
        if pattern.match(text):
            if not text.endswith(".TO"):
                text = text + ".TO"
            tickers.add(text)
    return list(tickers)

def get_all_tickers(limit=250):
    am_tickers = scrape_american_etf_tickers()
    ca_tickers = scrape_canadian_etf_tickers()
    all_tickers = list(set(am_tickers) | set(ca_tickers))
    print("Total tickers found:", len(all_tickers))
    if len(all_tickers) > limit:
        all_tickers = all_tickers[:limit]
    return all_tickers

# --- Fonctions de téléchargement et de traitement des données ---

def fetch_etf_data(ticker, start_date="2010-01-01", end_date=None, pause=1):
    print(f"Downloading data for {ticker} from {start_date}...")
    try:
        df = yf.download(ticker, start=start_date, end=end_date, progress=False)
        if not df.empty:
            df.reset_index(inplace=True)
            time.sleep(pause)  # Pour éviter de surcharger yfinance
            return df
        else:
            print(f"No data found for {ticker}.")
            return None
    except Exception as e:
        print(f"Error downloading data for {ticker}: {e}")
        return None

def process_historical_data(df):
    """
    Convertit le DataFrame (tel que retourné par yfinance) en chaîne JSON.
    La colonne de date est renommée en "Date_".
    """
    if df is None or df.empty:
        return None

    if isinstance(df.columns, pd.MultiIndex):
        df.columns = ['_'.join(map(str, col)) for col in df.columns.values]
    else:
        df.columns = df.columns.astype(str)

    if "Date_" not in df.columns:
        if 'date' in df.columns:
            df.rename(columns={'date': 'Date_'}, inplace=True)
        elif 'Date' in df.columns:
            df.rename(columns={'Date': 'Date_'}, inplace=True)

    # Convertir la colonne "Date_" en chaîne de caractères
    if 'Date_' in df.columns:
        df['Date_'] = df['Date_'].apply(lambda x: x.isoformat() if hasattr(x, 'isoformat') else str(x))

    data_list = df.to_dict(orient='records')
    return json.dumps(data_list, default=lambda o: o.isoformat() if hasattr(o, 'isoformat') else str(o))

def update_existing_history(existing_json, new_json):
    """
    Combine les données historiques existantes (JSON) avec les nouvelles (JSON).
    Les doublons sont supprimés sur la clé "Date_" en gardant le dernier enregistrement.
    """
    try:
        existing_list = json.loads(existing_json) if existing_json else []
    except Exception as e:
        print(f"Error parsing existing JSON: {e}")
        existing_list = []
    try:
        new_list = json.loads(new_json) if new_json else []
    except Exception as e:
        print(f"Error parsing new JSON: {e}")
        new_list = []
    combined = existing_list + new_list
    seen = {}
    for record in combined:
        if 'Date_' in record:
            seen[record['Date_']] = record
        else:
            print("Warning: record without 'Date_' encountered:", record)
    sorted_records = [seen[dt] for dt in sorted(seen)]
    return json.dumps(sorted_records, default=lambda o: o.isoformat() if hasattr(o, 'isoformat') else str(o))

# --- Traitement par ticker avec parallélisation ---

def process_ticker(ticker, existing_data, today):
    print(f"Processing ticker: {ticker}")
    if ticker in existing_data:
        try:
            existing_history = json.loads(existing_data[ticker])
        except Exception as e:
            print(f"Error parsing historical JSON for {ticker}: {e}")
            existing_history = []
        if existing_history:
            # Utiliser la clé "Date_" pour récupérer les dates
            dates = [record.get('Date_') for record in existing_history if record.get('Date_') is not None]
            if dates:
                last_date_str = max(dates)
            else:
                last_date_str = "2010-01-01"
            try:
                if 'T' in last_date_str:
                    last_date = datetime.strptime(last_date_str, "%Y-%m-%dT%H:%M:%S")
                else:
                    last_date = datetime.strptime(last_date_str, "%Y-%m-%d")
            except Exception as e:
                print(f"Error parsing date {last_date_str} for {ticker}: {e}")
                last_date = datetime(2010, 1, 1)
            new_start_date = (last_date + timedelta(days=1)).strftime("%Y-%m-%d")
            if last_date.date() >= today.date():
                print(f"Historical data for {ticker} is up-to-date.")
                return ticker, existing_data[ticker]
            else:
                print(f"Updating data for {ticker} starting from {new_start_date}...")
                df_new = fetch_etf_data(ticker, start_date=new_start_date)
                new_json = process_historical_data(df_new) if df_new is not None else None
                if new_json:
                    combined_json = update_existing_history(existing_data[ticker], new_json)
                    return ticker, combined_json
                else:
                    return ticker, existing_data[ticker]
        else:
            df_full = fetch_etf_data(ticker, start_date="2010-01-01")
            hist_json = process_historical_data(df_full) if df_full is not None else None
            return ticker, hist_json
    else:
        df_full = fetch_etf_data(ticker, start_date="2010-01-01")
        hist_json = process_historical_data(df_full) if df_full is not None else None
        return ticker, hist_json

def main():
    output_folder = "stocks"
    os.makedirs(output_folder, exist_ok=True)
    # Obtenir la date d'aujourd'hui au format AAAAMMJJ (exemple: 20250319)
    today = datetime.today().strftime('%Y%m%d')

    # Créer le nom de fichier en ajoutant la date avant l'extension
    filename = f"etf_historical_data_{today}.csv"

    # Construire le chemin complet
    output_file = os.path.join("stocks", filename)
    
    # Charger les données existantes si disponibles
    if os.path.exists(output_file):
        existing_df = pd.read_csv(output_file)
        existing_data = {row["Ticker"]: row["Historical"] for idx, row in existing_df.iterrows()}
        print(f"Found existing data for {len(existing_data)} tickers.")
    else:
        existing_data = {}
    
    tickers = get_all_tickers(limit=250)
    print(f"Total unique tickers to process: {len(tickers)}")
    updated_data = {}
    today = datetime.today()

    # Paralléliser le traitement des tickers (par exemple avec 10 workers)
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        future_to_ticker = {
            executor.submit(process_ticker, ticker, existing_data, today): ticker
            for ticker in tickers
        }
        for future in concurrent.futures.as_completed(future_to_ticker):
            ticker, hist_json = future.result()
            if hist_json is not None:
                updated_data[ticker] = hist_json

    final_df = pd.DataFrame([
        {"Ticker": ticker, "Historical": hist_json} for ticker, hist_json in updated_data.items()
    ])
    final_df.to_csv(output_file, index=False, encoding="utf-8")
    print(f"Final merged data saved to {output_file}")

if __name__ == "__main__":
    main()


Found existing data for 243 tickers.
Scraping American ETF tickers from: https://en.wikipedia.org/wiki/List_of_American_exchange-traded_funds
Scraping Canadian ETF tickers from: https://en.wikipedia.org/wiki/List_of_Canadian_exchange-traded_funds



1 Failed download:
['QEF.TO']: YFTzMissingError('possibly delisted; no timezone found')


Total tickers found: 430
Total unique tickers to process: 250
Processing ticker: GXF.TO
Processing ticker: HGY.TO
Updating data for HGY.TO starting from 2025-03-08...
Processing ticker: QEM.TO
Updating data for QEM.TO starting from 2024-05-29...
Processing ticker: SYLD.TO
Processing ticker: HCA.TO
Updating data for HCA.TO starting from 2025-03-08...
Processing ticker: PEU.TO
Updating data for PEU.TO starting from 2019-08-03...
Processing ticker: QEF.TO
Processing ticker: BND.TO
Processing ticker: ZFS.TO
Processing ticker: PPS.TO
No data found for HGY.TO.
Processing ticker: PBI.TO
Updating data for PBI.TO starting from 2025-03-08...



1 Failed download:

1 Failed download:
['CJP.TO']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-03-08 -> 2025-03-09)')

1 Failed download:

1 Failed download:
['CJP.TO']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-03-08 -> 2025-03-09)')

1 Failed download:


Processing ticker: EXM.TOProcessing ticker: RWC.TO

Processing ticker: HIX.TO
Processing ticker: QXM.TO
Processing ticker: VXC.TO
Processing ticker: XMU.TO
Processing ticker: XCR.TO
Processing ticker: RQH.TO
Processing ticker: HAC.TO
Updating data for HAC.TO starting from 2025-03-08...
Processing ticker: CJP.TO
Updating data for CJP.TO starting from 2025-03-08...
No data found for CJP.TO.
Processing ticker: XRE.TO
Error downloading data for EXM.TO: No objects to concatenate
Processing ticker: ZLU.TO
Error downloading data for QXM.TO: No objects to concatenate
Processing ticker: ZID.TO
Updating data for ZID.TO starting from 2025-03-08...


...

Processing ticker: ZEM.TOProcessing ticker: HUF.TO

Updating data for ZEM.TO starting from 2025-03-08...
Processing ticker: BXF.TO
Updating data for BXF.TO starting from 2025-03-08...
Updating data for HUF.TO starting from 2024-03-29...
Processing ticker: CHI.TO
Updating data for CHI.TO starting from 2015-08-28...
Processing ticker: XST.TO
Processing ticker: ZUQ.TO
Updating data for ZUQ.TO starting from 2025-03-08...
Processing ticker: CPD.TO
Updating data for CPD.TO starting from 2025-03-08...
Processing ticker: CBQ.TO
Processing ticker: FHC.TO
Processing ticker: PFH.TO
Updating data for PFH.TO starting from 2017-02-25...
No data found for PFH.TO.
Processing ticker: RHI.TO
Error downloading data for BXF.TO: No objects to concatenate
Processing ticker: HUC.TO
Error downloading data for ZEM.TO: No objects to concatenate
Processing ticker: MXF.TO
Updating data for MXF.TO starting from 2025-03-08...
Error downloading data for CPD.TO: No objects to concatenate
Processing ticker: HEF.TO



1 Failed download:
['HFD.TO']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-03-08 -> 2025-03-09)')

2 Failed downloads:
['HFD.TO', 'ZXM.TO']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-03-08 -> 2025-03-09)')

2 Failed downloads:

2 Failed downloads:

2 Failed downloads:

2 Failed downloads:

2 Failed downloads:

2 Failed downloads:

2 Failed downloads:
['HFD.TO', 'ZXM.TO']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-03-08 -> 2025-03-09)')
['HFD.TO', 'ZXM.TO']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-03-08 -> 2025-03-09)')
['HFD.TO', 'ZXM.TO']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-03-08 -> 2025-03-09)')
['HFD.TO', 'ZXM.TO']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-03-08 -> 2025-03-09)')
['HFD.TO', 'ZXM.TO']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-03-08 -> 2025-03-09)')
['HFD.TO',

Processing ticker: PZW.TO
Processing ticker: HBU.TO
Processing ticker: ZWB.TO
Processing ticker: PFG.TO
Processing ticker: HUZ.TO
Processing ticker: ZXM.TO
Updating data for ZXM.TO starting from 2025-03-08...
Processing ticker: TXF.TO
Updating data for TXF.TO starting from 2025-03-08...
Processing ticker: ITOT
Updating data for ITOT starting from 2025-03-08...
Processing ticker: PIN.TO
Updating data for PIN.TO starting from 2025-03-08...
Processing ticker: HFD.TO
Updating data for HFD.TO starting from 2025-03-08...



1 Failed download:

1 Failed download:

1 Failed download:
['TSX.TO']: YFTzMissingError('possibly delisted; no timezone found')
['TSX.TO']: YFTzMissingError('possibly delisted; no timezone found')
['TSX.TO']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:

1 Failed download:

1 Failed download:
['TSX.TO']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:

1 Failed download:
['TSX.TO']: YFTzMissingError('possibly delisted; no timezone found')
['TSX.TO']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['TSX.TO']: YFTzMissingError('possibly delisted; no timezone found')
['TSX.TO']: YFTzMissingError('possibly delisted; no timezone found')
['TSX.TO']: YFTzMissingError('possibly delisted; no timezone found')


Processing ticker: PGL.TO
Updating data for PGL.TO starting from 2017-12-14...
Processing ticker: TSX.TO
Processing ticker: FHB.TO
Updating data for FHB.TO starting from 2021-04-29...
Processing ticker: PXS.TO
Updating data for PXS.TO starting from 2025-03-08...
Processing ticker: BASE.TO
Updating data for BASE.TO starting from 2025-03-08...
Processing ticker: XCB.TO
Processing ticker: HFT.TO
Updating data for HFT.TO starting from 2024-01-19...
Processing ticker: ZEF.TO
Updating data for ZEF.TO starting from 2025-03-08...
Processing ticker: HAD.TO
No data found for PXS.TO.
No data found for BASE.TO.
No data found for XCB.TO.
No data found for HFT.TO.
No data found for PGL.TO.
No data found for ZEF.TO.
No data found for TSX.TO.
No data found for FHB.TO.
No data found for HAD.TO.
Final merged data saved to stocks/etf_historical_data.csv
